In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
from PIL import Image
Image.LOAD_TRUNCATED_IMAGES = True
import tensorflow as tf
from keras.preprocessing import image
from keras.callbacks import EarlyStopping
import keras
from keras import backend as K
from keras.applications.xception import preprocess_input
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
# getting files in the folder
files=[]
input_folder='foodinput'

# files in the folder 
for file in os.listdir(input_folder):
    files.append(file)

In [3]:
# getting the images from the folders
data_list=[]
input_path=input_folder+'/'
for item in files:
    data_list.append(os.listdir(input_path+item+'/'))

In [4]:
# grab images and converting to a np array 
def grab_image(img_path):
    img = image.load_img(img_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x) # returns values from o to 1
    return x

X_tr = []
y = []
for label in range(len(data_list)):
    #print(files[label])
    for pic_name in data_list[label]:
        y = np.append(y, label)
        X_tr.append(grab_image(input_path+files[label]+'/'+ pic_name))

        
X = np.concatenate(X_tr)
Y = keras.utils.to_categorical(y)
print(X.shape)
print(Y.shape)

(4000, 299, 299, 3)
(4000, 4)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3200, 299, 299, 3)
(800, 299, 299, 3)
(3200, 4)
(800, 4)


In [6]:
# save to csv file
#np.save('X_train', X_train)
#np.save('X_test', X_test)
#np.save('y_train', y_train)
#np.save('y_test', y_test)

#loaded_array = np.load('X_train.npy')

#The files created where quite huge so we are not creating them anymore

In [7]:
# Displaying pictures 
#apple_fig, apple_axes = plt.subplots(3,3)
#apple_fig.set_figheight(24)
#apple_fig.set_figwidth(24)

#for i, axis in enumerate(apple_axes):
#    for j, f in enumerate(axis):
#        img = plt.imread(apple+training_apple[i*3+j])
#        apple_axes[i,j].imshow(img)
#        apple_axes[i,j].set_title("Shape.{}".format(img.shape))

In [17]:
# very important 
input_shape = (299,299,3)
# number of classes 
num_classes = 101

In [18]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import load_model
from keras.applications.mobilenet import MobileNet

In [19]:
# MobileNet model, with weights pre-trained on ImageNet.
mobilenet = MobileNet(input_shape, weights='imagenet', include_top=False)

In [20]:
model = Sequential()
model.add(mobilenet)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()
sgd = SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Model)   (None, 9, 9, 1024)        3228864   
_________________________________________________________________
flatten_2 (Flatten)          (None, 82944)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               21233920  
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 1028      
Total params: 24,463,812
Trainable params: 24,441,924
Non-trainable params: 21,888
_________________________________________________________________


Now we need to prepare the training and testing dataset, we assign dogs to class 0 and cats to class 1.

In [21]:
# Remove single-dimensional entries from the shape of an array.
# data_train = y_train
# data_test = y_test
# label_train = X_train
# label_test = X_test

data_train = X_train
data_test = X_test
label_train = y_train
label_test = y_test

print("Training label shape: ", label_train.shape)
print("Testing label shape: ", label_test.shape)
print("Training data shape: ", data_train.shape)
print("Testing data shape: ", data_test.shape)

Training label shape:  (3200, 4)
Testing label shape:  (800, 4)
Training data shape:  (3200, 299, 299, 3)
Testing data shape:  (800, 299, 299, 3)


In [22]:
# early stop
early_stopping_monitor = EarlyStopping(patience=3)


#history = model.fit(data_train, label_train, 
#                    validation_split=0.2, 
#                    batch_size=4, epochs=10, 
#                    callbacks=[early_stopping_monitor])

Now start traning

In [ ]:
# train model
history = model.fit(data_train, label_train, validation_split=0.2, batch_size=4, epochs=10)

Train on 2560 samples, validate on 640 samples
Epoch 1/10
2560/2560 [==============================] - 108s 42ms/step - loss: 0.5416 - accuracy: 0.7744 - val_loss: 0.2584 - val_accuracy: 0.8973
Epoch 2/10
2560/2560 [==============================] - 100s 39ms/step - loss: 0.2984 - accuracy: 0.8683 - val_loss: 0.1835 - val_accuracy: 0.9277
Epoch 3/10
2560/2560 [==============================] - 100s 39ms/step - loss: 0.2105 - accuracy: 0.9142 - val_loss: 0.1570 - val_accuracy: 0.9301
Epoch 4/10
2560/2560 [==============================] - 100s 39ms/step - loss: 0.1760 - accuracy: 0.9322 - val_loss: 0.1464 - val_accuracy: 0.9391
Epoch 5/10
2560/2560 [==============================] - 100s 39ms/step - loss: 0.1292 - accuracy: 0.9495 - val_loss: 0.1476 - val_accuracy: 0.9359
Epoch 6/10
1924/2560 [=====================>........] - ETA: 23s - loss: 0.1065 - accuracy: 0.9605

In [ ]:
# save model
model.save("food_model3.h5")

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


Now we would like to see the performance of the model during training

In [ ]:
print(history.history.keys())


# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
test_result = model.predict(data_test, batch_size=4)

In [ ]:
test_result[1]

In [ ]:
_fig, _axes = plt.subplots(4,4)

_fig.set_figheight(24)
_fig.set_figwidth(24)


for i in range(4):
    for j in range(4):
        img = data_test[i*3+j]
        _axes[i,j].imshow(img)
        if np.argmax(test_result[i*3+j]) == 0:
            _axes[i,j].set_title("Prediction  {}".format('apple pie'))
        elif np.argmax(test_result[i*3+j]) == 1:
            _axes[i,j].set_title("Prediction  {}".format('waffle'))
        elif np.argmax(test_result[i*3+j]) == 2:
            _axes[i,j].set_title("Prediction  {}".format('churro')) 

Now let's plot some of the wrong classifications